In [6]:
# Use two packages to open Excel files
#!pip install xlrd --break-system-packages

#!pip install openpyxl --break-system-packages

# Install the disambiguation package (no need for a master list of firm names)
#pip install disamby --break-system-packages
#!pip install rapidfuzz

# Load package for data manipulation purposes
import pandas as pd
import re
from rapidfuzz import fuzz, process

In [7]:
# Load in June 2015 data to make a complete set of data; get rid of associations that list condo in name and those with zero listed units

june15_df = pd.read_excel("../Data/NRED/2015/NRED HOA - 06 2015.xls")
june15_df = june15_df[~june15_df['Name'].str.contains('condo', case=False, na=False)]
june15_df = june15_df[~(june15_df['# of Units'] == 0)]

june15_df 

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County
0,1200 RIVERSIDE DRIVE ASSOCIATION,283-1972,REG,100,(702) 737-8580,"C/O FIRSTSERVICE RESIDENTIAL, NEVADA, LLC",8290 ARVILLE ST,LAS VEGAS,NV,89139,Washoe
1,338 MILKY WAY COURT HOMEOWNERS ASSOCIATION INC...,0208852006-1,SUB,4,(917) 208-0950,NaN,447 W 45TH ST STE 4E,NEW YORK,NY,10036,NaN
2,344 QUAKING ASPEN LANE ASSOCIATION,10679-1991,SUB,4,(775) 230-3620,NaN,PO BOX 5486,STATELINE,NV,89449,Douglas
3,"389 COTTONWOOD HOMEOWNERS ASSOCIATION, A NEVAD...",30976-2003,REG,2,(707) 642-2680,NaN,154 SKYWAY,VALLEJO,CA,94591,NaN
4,391 WILLOW COURT HOMEOWNERS ASSOCIATION,5719-1978,REG,4,(775) 831-5345,C/O SWISS TIME MANAGEMENT,PO BOX 5742,INCLINE VILLAGE,NV,89450,Washoe
...,...,...,...,...,...,...,...,...,...,...,...
3227,YORK VILLAGE COMMUNITY ASSOCIATION,0679632006-4,REG,127,(702) 655-7064,C/O LAS VEGAS VALLEY COMMUNITY MANAGEMENT,7571 TULE SPRINGS RD,LAS VEGAS,NV,89131,Clark
3228,YORKSHIRE MANOR ASSOCIATION,2411-1971,REG,160,(775) 674-8000,C/O KENYON & ASSOCIATES,645 SIERRA ROSE DR STE 105A,RENO,NV,89511,Washoe
3229,YORKSHIRE MANOR NO.2 ASSOCIATION,3248-1972,REG,108,(775) 674-8000,C/O KENYON & ASSOCIATES,645 SIERRA ROSE DR STE 105A,RENO,NV,89511,Washoe
3230,ZEPHYR RIDGE HOMEOWNERS' ASSOCIATION,0553902013-5,REG,88,(702) 531-3382,C/O CAMCO,PO BOX 12117,LAS VEGAS,NV,89112,Clark


In [8]:
# Create a subset of observations that have management companies associated with them 
# Then sort in ascending alphabetical order according to the management company names
june15_subset = june15_df[june15_df['Address1'].str.contains('C/O', na=False)].sort_values(by='Address1')
june15_subset['City'] = june15_subset['City'].str.title()
june15_subset['Address1'] = june15_subset['Address1'].str.replace(r'\bC/O\b', '', regex=True).str.title()
june15_subset['Address2'] = june15_subset['Address2'].str.title()
june15_subset['Name'] = june15_subset['Name'].str.title()

june15_subset

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County
2752,Sutton Place Homeowners Association,1190-1985,REG,112,(702) 943-0981,"5 Star Management, Llc",8871 W Flamingo Rd Ste 202,Las Vegas,NV,89147,Clark
251,Bonanza Park Homeowners Association,1888-1983,REG,132,(702) 943-0981,5 Str Mgt Llc,8871 W Flamingo Rd Ste 202,Las Vegas,NV,89147,Clark
934,"Evans Ridge Estates Association, Inc.",9360-2000,REG,61,(775) 560-2202,"Abc Management, Llc",6015 S Virginia St E267,Reno,NV,89502,Washoe
1316,Jonathan'S Glen Iii Homeowners' Association,3211-1991,REG,42,(702) 645-1210,Adept Management Services Inc,5550 Painted Mirage Rd Ste 320,Las Vegas,NV,89149,Clark
460,Cedar Springs Homeowners Association,29739-2003,REG,103,(702) 645-1210,Adept Management Services Inc,5550 Painted Mirage Rd Ste 320,Las Vegas,NV,89149,Clark
...,...,...,...,...,...,...,...,...,...,...,...
1336,Knollcrest Homeowners Association,16308-1994,REG,13,(775) 284-4434,"Western Nevada Management, Inc.",804 Mill St,Reno,NV,89502,Washoe
1396,Lake-Ridge Shores Townhouses Association,6872-1982,SUB,28,(775) 284-4434,"Western Nevada Properties, Inc",804 Mills Street,Reno,NV,89502,Washoe
101,Arlington Towers Homeowners Association,5587-1980,REG,194,(775) 284-4434,"Western Nevada Properties, Inc",804 Mill St,Reno,NV,89502,Washoe
1927,Park Terrace Townhomes Association,1865-1984,REG,182,(775) 284-4434,"Western Nevada Properties, Inc",804 Mill St,Reno,NV,89502,Washoe


In [9]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

def singularize_word(word):
    if len(word) > 3 and word.endswith('s') and not word.endswith('ss'):
        return word[:-1]
    return word

def normalize_address(address):
    if pd.isnull(address):
        return ''
    address = address.lower()
    
    # Normalize common road types
    address = re.sub(r'\b(streets|street|st)\b', 'st', address)
    address = re.sub(r'\b(roads|road|rd)\b', 'rd', address)
    address = re.sub(r'\b(avenues|avenue|ave)\b', 'ave', address)
    address = re.sub(r'\b(boulevards|boulevard|blvd)\b', 'blvd', address)
    address = re.sub(r'\b(suites|suite|ste|se)\b', 'ste', address) 
    address = re.sub(r'\b(apartments|apartment|apt)\b', 'apt', address)
    address = re.sub(r'\b(forts|fort|ft)\b', 'ft', address)
    
    # Standardize street directions
    address = re.sub(r'\b(south|s)\b', 's', address)
    address = re.sub(r'\b(north|n)\b', 'n', address)
    address = re.sub(r'\b(east|e)\b', 'e', address)
    address = re.sub(r'\b(west|w)\b', 'w', address)
    address = re.sub(r'\b(southeast|se)\b', 'se', address)
    address = re.sub(r'\b(northeast|ne)\b', 'ne', address)
    address = re.sub(r'\b(southwest|sw)\b', 'sw', address)
    address = re.sub(r'\b(northwest|nw)\b', 'nw', address)
    
    # Eliminate firm office indicators
    address = re.split(r'\b(suite|ste|apt|apartment|unit|site)\b', address)[0]
    
    # Remove punctuation and number sign
    address = re.sub(r'[.,]', '', address)
    address = re.sub(r'\s+', ' ', address).strip()
    address = re.sub(r'#', '', address)

    # Singularize words
    address = ' '.join(singularize_word(word) for word in address.split())

    return address

# Do a similar process for the firm names
def normalize_firm_name(name):
    if pd.isnull(name):
        return ''
    name = name.lower().strip()
    
    # Remove suffixes
    name = re.sub(r'\b(llc|inc|corp|co|ltd)\b', '', name)
    
    # Remove commas etc.
    name = re.sub(r'[^\w\s]', '', name)
    
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name)
    
    # Singularize words
    name = ' '.join(singularize_word(word) for word in name.split())
    
    return name

# Normalize the phone
def normalize_phone(phone):
    if pd.isnull(phone):
        return ''
    return re.sub(r'\D', '', str(phone))

def get_mode(series):
    mode = series.mode()
    return mode.iloc[0] if not mode.empty else series.iloc[0]

june15_subset['Address2_Normalized'] = june15_subset['Address2'].apply(normalize_address)
june15_subset['Firm_Normalized'] = june15_subset['Address1'].apply(normalize_firm_name)
june15_subset['Phone_Normalized'] = june15_subset['Telephone'].apply(normalize_phone)

june15_subset['Firm_Normalized'] = june15_subset['Firm_Normalized'].str.replace(
    r'\bmangement\b', 'management', regex=True
)

# Most common phone and address by firm name
canonical_phone = (
    june15_subset.groupby('Phone_Normalized')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized': 'Firm_ByPhone'})
    .reset_index()
)

canonical_address = (
    june15_subset.groupby('Address2_Normalized')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized': 'Firm_ByAddress'})
    .reset_index()
)

june15_subset = june15_subset.merge(canonical_phone, on='Phone_Normalized', how='left')
june15_subset = june15_subset.merge(canonical_address, on='Address2_Normalized', how='left')

# Build the hierarchy
def pick_canonical(row):
    if pd.notnull(row['Firm_ByPhone']):
        return row['Firm_ByPhone']
    elif pd.notnull(row['Firm_ByAddress']):
        return row['Firm_ByAddress']
    else:
        return row['Firm_Normalized']

june15_subset['Firm_Canonical'] = june15_subset.apply(pick_canonical, axis=1)

# Fuzzy matching within the canonical deduplication
def fuzzy_within_group(group, threshold=90):
    unique_names = group['Firm_Canonical'].unique()
    name_map = {}
    for name in unique_names:
        if name in name_map:
            continue
        matches = process.extract(name, unique_names, scorer=fuzz.token_set_ratio, limit=None)
        close_matches = [m[0] for m in matches if m[1] >= threshold]
        for cm in close_matches:
            name_map[cm] = name
    group['Firm_Final'] = group['Firm_Canonical'].map(name_map).fillna(group['Firm_Canonical'])
    return group

# Apply fuzzy matching **within each phone group first** to avoid cross-company merges
june15_subset = june15_subset.groupby('Phone_Normalized').apply(fuzzy_within_group).reset_index(drop=True)

# Fuzzy across all names to catch edge cases
all_names = june15_subset['Firm_Final'].unique()
name_map_global = {}

for name in all_names:
    if name in name_map_global:
        continue
    matches = process.extract(name, all_names, scorer=fuzz.token_set_ratio, limit=None)
    close_matches = [m[0] for m in matches if m[1] >= 90]  
    for cm in close_matches:
        name_map_global[cm] = name

june15_subset['Firm_Final'] = june15_subset['Firm_Final'].map(name_map_global).fillna(june15_subset['Firm_Final'])

# Save data
june15_subset.to_csv('../Data/Cleaned files/june15_subset.csv', index=False)

In [14]:
# Load in June 2025 data to make a complete set of data; get rid of associations that list condo in name and zero units cases

june25_df = pd.read_excel("../Data/NRED/2025/NRED HOA - 06 2025.xlsx")
june25_df = june25_df[~june25_df['Name'].str.contains('condo', case=False, na=False)]
june25_df = june25_df[~(june25_df['# of Units'] == 0)]

june25_df 

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County
0,1200 RIVERSIDE DRIVE ASSOCIATION,283-1972,REG,100.0,(775) 674-8000,KENYON & ASSOCIATES,645 SIERRA ROSE DR STE 104,RENO,NV,89511,Washoe
1,"14TH STREET DUPLEX HOMEOWNERS ASSOCIATION, INC",12771462021-4,REG,2.0,(702) 290-2122,"C/O 14TH STREET DUPLEX HOMEOWNERS ASSOCIATION,...",9292 BLUEMIST FALLS ST,LAS VEGAS,NV,89123,Clark
2,"305 GALAXY HOMEOWNERS ASSOCIATION, INC",49255802025-9,SUB,9.0,(775) 901-1088,NaN,"ATTN: MICHAEL L. MATUSKA, ESQ",CARSON CITY,NV,89701,Carson City
3,317 QUAKING ASPEN HOMEOWNERS ASSOCIATION,0100782019-8,SUB,8.0,(775) 284-4434,"C/O WESTERN NEVADA MANAGEMENT, INC","255 W. PECKHAM LN., STE 2",RENO,NV,89509,Douglas
4,338 MILKY WAY COURT HOMEOWNERS ASSOCIATION INC...,0208852006-1,SUB,4.0,(412) 952-6509,NaN,CMR 480 BOX 1769,APO,AB,9128,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3892,YORK VILLAGE COMMUNITY ASSOCIATION,0679632006-4,REG,127.0,(702) 655-7064,C/O NEVADA COMMUNITY MANAGEMENT,9500 WEST FLAMINGO RD STE 204,LAS VEGAS,NV,89147,Clark
3893,YORKSHIRE MANOR ASSOCIATION,2411-1971,REG,160.0,(775) 284-4434,"C/O WESTERN NEVADA MANAGEMENT, INC",255 W PECKHAM LN STE 2,RENO,NV,89509,Washoe
3894,YORKSHIRE MANOR NO.2 ASSOCIATION,3248-1972,REG,108.0,(775) 446-4479,C/O HOAMCO,5484 RENO CORPORATE DRIVE #100,RENO,NV,89511,Washoe
3895,ZEPHYR RIDGE HOMEOWNERS' ASSOCIATION,0553902013-5,REG,88.0,(702) 362-6262,C/O TERRA WEST MANAGEMENT SERVICES,6655 S CIMARRON RD STE 200,LAS VEGAS,NV,89113,Clark


In [15]:
# Create another subset
june25_subset = june25_df[june25_df['Address1'].str.contains('C/O', na=False)].sort_values(by='Address1')
june25_subset['City'] = june25_subset['City'].str.title()
june25_subset['Address1'] = june25_subset['Address1'].str.replace(r'\bC/O\b', '', regex=True).str.title()
june25_subset['Address2'] = june25_subset['Address2'].str.title()
june25_subset['Name'] = june25_subset['Name'].str.title()

june25_subset

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County
1,"14Th Street Duplex Homeowners Association, Inc",12771462021-4,REG,2.0,(702) 290-2122,"14Th Street Duplex Homeowners Association, Inc",9292 Bluemist Falls St,Las Vegas,NV,89123,Clark
11,"428 Quaking Aspen Hoa, Llc",0617562006-1,SUB,4.0,(775) 690-0808,"428 Quaking Aspen Hoa, Llc",16310 Jeffe Ct,Reno,NV,89511,Washoe
1837,Linkview Drive Homeowners Association Inc,10041-1989,SUB,98.0,(702) 645-1210,Adept Management Services,7495 W Azure Dr Ste 140,Las Vegas,NV,89130,Clark
773,Coronado Ranch 5 Homeowners Association,0433482018-3,REG,324.0,(702) 606-3330,Allied Association Advisors Llc,"9205 W Russell Rd., Ste 240",Las Vegas,NV,89148,Clark
1875,Lorryn Parc Homeowners Association,1168-2005,REG,20.0,(702) 606-3330,Allied Association Advisors Llc,"9205 W Russell Rd., Ste 240",Las Vegas,NV,89148,Clark
...,...,...,...,...,...,...,...,...,...,...,...
59,Allure I Townhomes Homeowners Association,3232-1994,REG,126.0,(702) 953-2226,Westward360,3265 N Fort Apache Rd Ste 160,Las Vegas,NV,89129,Clark
1132,Equestrian Court Homeowners Association,28017-2001,REG,100.0,(702) 953-2226,Westward360 Inc,"3265 N Fort Apache Road, Suite 160",Las Vegas,NV,89129,Clark
2111,Monterosso Premier Homeowners Association,24282-2000,SUB,91.0,(702) 953 -2226,Westward360 Inc,3265 N Fort Apache Rd Ste 160,Las Vegas,NV,89129,Clark
3591,Valley Del Paradiso Homeowners Association,2325-1983,REG,96.0,(702) 953-1738,"Westward360, Inc",3265 N Ft Apache Rd Ste 160,Las Vegas,NV,89129,Clark


In [16]:
# Repeat process for second static period

june25_subset['Address2_Normalized'] = june25_subset['Address2'].apply(normalize_address)
june25_subset['Firm_Normalized'] = june25_subset['Address1'].apply(normalize_firm_name)
june25_subset['Phone_Normalized'] = june25_subset['Telephone'].apply(normalize_phone)

june25_subset['Firm_Normalized'] = june25_subset['Firm_Normalized'].str.replace(
    r'\bmangement\b', 'management', regex=True
)

canonical_phone = (
    june25_subset.groupby('Phone_Normalized')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized': 'Firm_ByPhone'})
    .reset_index()
)

canonical_address = (
    june25_subset.groupby('Address2_Normalized')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized': 'Firm_ByAddress'})
    .reset_index()
)

june25_subset = june25_subset.merge(canonical_phone, on='Phone_Normalized', how='left')
june25_subset = june25_subset.merge(canonical_address, on='Address2_Normalized', how='left')

june25_subset['Firm_Canonical'] = june25_subset.apply(pick_canonical, axis=1)

june25_subset = june25_subset.groupby('Phone_Normalized').apply(fuzzy_within_group).reset_index(drop=True)

all_names = june25_subset['Firm_Final'].unique()
name_map_global = {}

for name in all_names:
    if name in name_map_global:
        continue
    matches = process.extract(name, all_names, scorer=fuzz.token_set_ratio, limit=None)
    close_matches = [m[0] for m in matches if m[1] >= 90] 
    for cm in close_matches:
        name_map_global[cm] = name

june25_subset['Firm_Final'] = june25_subset['Firm_Final'].map(name_map_global).fillna(june25_subset['Firm_Final'])

june25_subset.to_csv('../Data/Cleaned files/june25_subset.csv', index=False)

In [26]:
!pip install pyxlsb

import pandas as pd
import glob
import os

nred_folder = '../Data/NRED/'

# Find all Excel files within the folders
all_files = glob.glob(os.path.join(nred_folder, '**', '*.xls*'), recursive=True)
print(f"Found {len(all_files)} Excel files")

# Create new list to store
month_data = []

for file in all_files:
    try:
        # Read Excel, handle .xlsb and .xls/.xlsx
        if file.endswith('.xlsb'):
            df = pd.read_excel(file, engine='pyxlsb')
        else:
            df = pd.read_excel(file)
        
        if df.empty:
            continue

        # Try to extract month and year from file name (safer)
        # Example file name: "NRED HOA - 06 2015.xls"
        base_name = os.path.splitext(os.path.basename(file))[0]
        parts = base_name.split('-')[-1].strip().split()
        if len(parts) != 2:
            print(f"Skipping {file}, cannot extract month/year")
            continue
        month = int(parts[0])
        year = int(parts[1])

        # Filter out condos
        df = df[~df['Name'].str.contains('condo', case=False, na=False)]

        # Filter zero units using proper column name
        if '# of Units' in df.columns:
            df = df[df['# of Units'] > 0]

        # Count total and professionally managed
        total_rows = df.shape[0]
        professional_rows = df['Address1'].str.contains('C/O', case=False, na=False).sum()

        month_data.append({
            'Year': year,
            'Month': month,
            'Total_HOAs': total_rows,
            'Professional_HOAs': professional_rows
        })

    except Exception as e:
        print(f"Error reading {file}: {e}")

if not month_data:
    raise ValueError("No valid files found to process!")

# Convert to DataFrame
summary_df = pd.DataFrame(month_data)

# Sort by Year and Month
summary_df = summary_df.sort_values(['Year', 'Month']).reset_index(drop=True)

# Save for Stata
summary_df.to_csv('../Data/Misc/HOA_summary_monthly_for_stata.csv', index=False)
print("Monthly summary CSV saved!")

Found 121 Excel files


C:\Users\19548\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Monthly summary CSV saved!


In [27]:
# Looping pipeline for all months using the June 2015/2025 format

import pandas as pd
import glob
import os
import re
from rapidfuzz import process, fuzz

def singularize_word(word):
    if len(word) > 3 and word.endswith('s') and not word.endswith('ss'):
        return word[:-1]
    return word

def normalize_address(address):
    if pd.isnull(address):
        return ''
    address = address.lower()
    address = re.sub(r'\b(streets|street|st)\b', 'st', address)
    address = re.sub(r'\b(roads|road|rd)\b', 'rd', address)
    address = re.sub(r'\b(avenues|avenue|ave)\b', 'ave', address)
    address = re.sub(r'\b(boulevards|boulevard|blvd)\b', 'blvd', address)
    address = re.sub(r'\b(suites|suite|ste|se)\b', 'ste', address)
    address = re.sub(r'\b(apartments|apartment|apt)\b', 'apt', address)
    address = re.sub(r'\b(forts|fort|ft)\b', 'ft', address)
    address = re.sub(r'\b(south|s)\b', 's', address)
    address = re.sub(r'\b(north|n)\b', 'n', address)
    address = re.sub(r'\b(east|e)\b', 'e', address)
    address = re.sub(r'\b(west|w)\b', 'w', address)
    address = re.sub(r'\b(southeast|se)\b', 'se', address)
    address = re.sub(r'\b(northeast|ne)\b', 'ne', address)
    address = re.sub(r'\b(southwest|sw)\b', 'sw', address)
    address = re.sub(r'\b(northwest|nw)\b', 'nw', address)
    address = re.split(r'\b(suite|ste|apt|apartment|unit|site)\b', address)[0]
    address = re.sub(r'[.,]', '', address)
    address = re.sub(r'\s+', ' ', address).strip()
    address = re.sub(r'#', '', address)
    address = ' '.join(singularize_word(word) for word in address.split())
    return address

def normalize_firm_name(name):
    if pd.isnull(name):
        return ''
    name = name.lower().strip()
    name = re.sub(r'\b(llc|inc|corp|co|ltd)\b', '', name)
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name)
    name = ' '.join(singularize_word(word) for word in name.split())
    return name

def normalize_phone(phone):
    if pd.isnull(phone):
        return ''
    return re.sub(r'\D', '', str(phone))

def get_mode(series):
    mode = series.mode()
    return mode.iloc[0] if not mode.empty else series.iloc[0]

def pick_canonical(row):
    if pd.notnull(row['Firm_ByPhone']):
        return row['Firm_ByPhone']
    elif pd.notnull(row['Firm_ByAddress']):
        return row['Firm_ByAddress']
    else:
        return row['Firm_Normalized']

def fuzzy_within_group(group, threshold=90):
    unique_names = group['Firm_Canonical'].unique()
    name_map = {}
    for name in unique_names:
        if name in name_map:
            continue
        matches = process.extract(name, unique_names, scorer=fuzz.token_set_ratio, limit=None)
        close_matches = [m[0] for m in matches if m[1] >= threshold]
        for cm in close_matches:
            name_map[cm] = name
    group['Firm_Final'] = group['Firm_Canonical'].map(name_map).fillna(group['Firm_Canonical'])
    return group

# Path to all files
root_folder = "../Data/NRED/"
output_folder = "../Data/Cleaned files/"
os.makedirs(output_folder, exist_ok=True)

# Recursively get all Excel files as I did in the Stata summary thing
excel_files = glob.glob(os.path.join(root_folder, '**', '*.xls*'), recursive=True)
print(f"Found {len(excel_files)} Excel files")

for file in excel_files:
    try:
        df = pd.read_excel(file, engine=None)  # let pandas infer engine
        # Filtering condos and zero-unit rows
        df = df[~df['Name'].str.contains('condo', case=False, na=False)]
        df = df[~(df['# of Units'] == 0)]
        # Subset for professional management
        subset = df[df['Address1'].str.contains('C/O', na=False)].copy()
        subset['City'] = subset['City'].str.title()
        subset['Address1'] = subset['Address1'].str.replace(r'\bC/O\b', '', regex=True).str.title()
        subset['Address2'] = subset['Address2'].str.title()
        subset['Name'] = subset['Name'].str.title()

        # Normalization
        subset['Address2_Normalized'] = subset['Address2'].apply(normalize_address)
        subset['Firm_Normalized'] = subset['Address1'].apply(normalize_firm_name)
        subset['Phone_Normalized'] = subset['Telephone'].apply(normalize_phone)
        subset['Firm_Normalized'] = subset['Firm_Normalized'].str.replace(
            r'\bmangement\b', 'management', regex=True
        )

        # Canonical by phone and address
        canonical_phone = (
            subset.groupby('Phone_Normalized')
            .agg({'Firm_Normalized': get_mode})
            .rename(columns={'Firm_Normalized': 'Firm_ByPhone'})
            .reset_index()
        )
        canonical_address = (
            subset.groupby('Address2_Normalized')
            .agg({'Firm_Normalized': get_mode})
            .rename(columns={'Firm_Normalized': 'Firm_ByAddress'})
            .reset_index()
        )
        subset = subset.merge(canonical_phone, on='Phone_Normalized', how='left')
        subset = subset.merge(canonical_address, on='Address2_Normalized', how='left')
        subset['Firm_Canonical'] = subset.apply(pick_canonical, axis=1)

        # Fuzzy matching (apply within group function first and then)
        subset = subset.groupby('Phone_Normalized').apply(fuzzy_within_group).reset_index(drop=True)
        all_names = subset['Firm_Final'].unique()
        name_map_global = {}
        for name in all_names:
            if name in name_map_global:
                continue
            matches = process.extract(name, all_names, scorer=fuzz.token_set_ratio, limit=None)
            close_matches = [m[0] for m in matches if m[1] >= 90]  
            for cm in close_matches:
                name_map_global[cm] = name
        subset['Firm_Final'] = subset['Firm_Final'].map(name_map_global).fillna(subset['Firm_Final'])

        # Save cleaned file
        base_name = os.path.splitext(os.path.basename(file))[0]
        subset.to_csv(os.path.join(output_folder, f"{base_name}_subset.csv"), index=False)
        print(f"Cleaned {file} -> {base_name}_subset.csv")

    except Exception as e:
        print(f"Error processing {file}: {e}")

Found 121 Excel files
Cleaned ../Data/NRED\2015\NRED HOA - 06 2015.xls -> NRED HOA - 06 2015_subset.csv
Cleaned ../Data/NRED\2015\NRED HOA - 07 2015.xlsx -> NRED HOA - 07 2015_subset.csv
Cleaned ../Data/NRED\2015\NRED HOA - 08 2015.xls -> NRED HOA - 08 2015_subset.csv
Cleaned ../Data/NRED\2015\NRED HOA - 09 2015.xls -> NRED HOA - 09 2015_subset.csv
Cleaned ../Data/NRED\2015\NRED HOA - 10 2015.xls -> NRED HOA - 10 2015_subset.csv
Cleaned ../Data/NRED\2015\NRED HOA - 11 2015.xls -> NRED HOA - 11 2015_subset.csv
Cleaned ../Data/NRED\2015\NRED HOA - 12 2015.xls -> NRED HOA - 12 2015_subset.csv
Cleaned ../Data/NRED\2016\NRED HOA - 01 2016.xls -> NRED HOA - 01 2016_subset.csv
Cleaned ../Data/NRED\2016\NRED HOA - 02 2016.xls -> NRED HOA - 02 2016_subset.csv
Cleaned ../Data/NRED\2016\NRED HOA - 03 2016.xls -> NRED HOA - 03 2016_subset.csv
Cleaned ../Data/NRED\2016\NRED HOA - 04 2016.xls -> NRED HOA - 04 2016_subset.csv
Cleaned ../Data/NRED\2016\NRED HOA - 05 2016.xls -> NRED HOA - 05 2016_subs

C:\Users\19548\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Cleaned ../Data/NRED\2018\NRED HOA - 07 2018.xlsx -> NRED HOA - 07 2018_subset.csv
Cleaned ../Data/NRED\2018\NRED HOA - 08 2018.xlsx -> NRED HOA - 08 2018_subset.csv
Cleaned ../Data/NRED\2018\NRED HOA - 09 2018.xlsx -> NRED HOA - 09 2018_subset.csv
Cleaned ../Data/NRED\2018\NRED HOA - 10 2018.xlsx -> NRED HOA - 10 2018_subset.csv
Cleaned ../Data/NRED\2018\NRED HOA - 11 2018.xlsx -> NRED HOA - 11 2018_subset.csv
Cleaned ../Data/NRED\2018\NRED HOA - 12 2018.xlsx -> NRED HOA - 12 2018_subset.csv
Cleaned ../Data/NRED\2019\NRED HOA - 01 2019.xlsx -> NRED HOA - 01 2019_subset.csv
Cleaned ../Data/NRED\2019\NRED HOA - 02 2019.xlsx -> NRED HOA - 02 2019_subset.csv
Cleaned ../Data/NRED\2019\NRED HOA - 03 2019.xlsx -> NRED HOA - 03 2019_subset.csv
Cleaned ../Data/NRED\2019\NRED HOA - 04 2019.xlsx -> NRED HOA - 04 2019_subset.csv
Cleaned ../Data/NRED\2019\NRED HOA - 05 2019.xlsx -> NRED HOA - 05 2019_subset.csv
Cleaned ../Data/NRED\2019\NRED HOA - 06 2019.xlsx -> NRED HOA - 06 2019_subset.csv
Clea